In [3]:
# import required modules
import os
import pandas as pd
import numpy as np
from sklearn import metrics

In [4]:
# set directory paths
root = '../../..'
result_csv = os.path.join(root, 'results/results_validation_fasterRCNN-07-27.csv')

In [5]:
# create dataframe
result_df = pd.read_csv(result_csv)

In [155]:
# add results column for confidence threshold -- work in progress
def add_conf_result_col (df, iou_thres, conf_thres):
    df['tp'] = np.where((df.iou > iou_thres) & (df.confidence > conf_thres), 1, 0)
    df['fp'] = np.where(((df.iou <= iou_thres) | (df.confidence <= conf_thres)) | ((df.pred_bbox.notnull() & df.gt_bbox.isnull())), 1, 0)
    df['fn'] = np.where(((df.iou <= iou_thres) | (df.confidence <= conf_thres)) | ((df.pred_bbox.isnull() & df.gt_bbox.notnull())), 1, 0)
    df['result@IoU'+str(iou_thres)+'@conf'+str(conf_thres)] = df.apply(
        lambda row: 'tp' if row.tp == 1 else 'fp+fn' if (row.fp == 1) & (row.fn == 1) else 'fp' if (row.fp == 1) else 'fn' if (row.fn == 1) else 'tn',
        axis=1
    )
    df = df.drop(['tp', 'fp', 'fn'], axis=1)
    return df

In [178]:
result_df = add_conf_result_col(result_df, 0.05, 0.7)
result_df

,img_id,gt_bbox,gt_format,pred_bbox,pred_format,confidence,gt_size,iou,result@IoU0.0,result@IoU0.05,result@IoU0.1,result@IoU0.2,result@IoU0.5,result@IoU0.05@conf0.3,result@IoU0.05@conf0.5,result@IoU0.05@conf0.7
0,2672.txt,[305 300 377 382],left/bottom/right/top,[354 302 312 382],top/left/bottom/right,0.8374,5904.0,0.484615,tp,tp,tp,tp,fp+fn,tp,tp,tp
1,125569.txt,[185 198 356 343],left/bottom/right/top,[348 189 218 376],top/left/bottom/right,0.8801,24795.0,0.739462,tp,tp,tp,tp,tp,tp,tp,tp
2,125730.txt,[106 94 246 258],left/bottom/right/top,[259 137 118 199],top/left/bottom/right,0.9283,22960.0,0.377031,tp,tp,tp,tp,fp+fn,tp,tp,tp
3,105933.txt,[314 321 357 350],left/bottom/right/top,[355 322 324 355],top/left/bottom/right,0.8858,1247.0,0.607649,tp,tp,tp,tp,tp,tp,tp,tp
4,8124.txt,[176 192 220 204],left/bottom/right/top,[153 121 128 138],top/left/bottom/right,0.1947,528.0,0.000000,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn
5,8382.txt,[105 127 187 165],left/bottom/right/top,[170 105 123 183],top/left/bottom/right,0.9389,3116.0,0.776323,tp,tp,tp,tp,tp,tp,tp,tp
6,124613.txt,[241 231 274 283],left/bottom/right/top,[285 240 233 279],top/left/bottom/right,0.4962,1716.0,0.787966,tp,tp,tp,tp,tp,tp,fp+fn,fp+fn
7,6270.txt,[154 169 216 200],left/bottom/right/top,[205 157 170 217],top/left/bottom/right,0.8589,1922.0,0.785968,tp,tp,tp,tp,tp,tp,tp,tp
8,113486.txt,[246 250 262 257],left/bottom/right/top,[273 247 262 263],top/left/bottom/right,0.2726,112.0,0.000000,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn
9,99656.txt,[106 130 198 174],left/bottom/right/top,[182 108 133 203],top/left/bottom/right,0.6457,4048.0,0.736086,tp,tp,tp,tp,tp,tp,tp,fp+fn


In [181]:
def create_result_sparse_df (result_df, iou_thres, conf_thres):
    
    fp_fn_df = result_df[result_df['result@IoU0.05'+'@conf'+str(conf_thres)] == 'fp+fn']
    fp_only_df = fp_fn_df.copy()
    fn_only_df = fp_fn_df.copy()
    fp_only_df['result@IoU0.05'+'@conf'+str(conf_thres)] = 'fp'
    fp_only_df['gt_bbox'] = None
    fn_only_df['result@IoU0.05'+'@conf'+str(conf_thres)] = 'fn'
    non_fp_fn_df = result_df[result_df['result@IoU0.05'+'@conf'+str(conf_thres)] != 'fp+fn']
    result_df_a = non_fp_fn_df.append(fp_only_df)
    result_df_sparse = result_df_a.append(fn_only_df)
    
    return result_df_sparse

#print(result_df_sparse.sort_values('img_id'))

In [185]:
result_df_sparse = create_result_sparse_df(result_df, 0.05, 0.5)
result_df_sparse.sort_values('img_id', ascending=False)

,img_id,gt_bbox,gt_format,pred_bbox,pred_format,confidence,gt_size,iou,result@IoU0.0,result@IoU0.05,result@IoU0.1,result@IoU0.2,result@IoU0.5,result@IoU0.05@conf0.3,result@IoU0.05@conf0.5,result@IoU0.05@conf0.7
1265,99998.txt,[214 278 386 322],left/bottom/right/top,[323 213 275 385],top/left/bottom/right,0.8560,7568.0,0.906506,tp,tp,tp,tp,tp,tp,tp,tp
167,99989.txt,[329 311 356 373],left/bottom/right/top,[374 329 314 359],top/left/bottom/right,0.8369,1674.0,0.846890,tp,tp,tp,tp,tp,tp,tp,tp
6168,99981.txt,[256 215 298 339],left/bottom/right/top,[369 269 212 327],top/left/bottom/right,0.2423,5208.0,0.335510,tp,tp,tp,tp,fp+fn,fp+fn,fn,fp+fn
6168,99981.txt,None,left/bottom/right/top,[369 269 212 327],top/left/bottom/right,0.2423,5208.0,0.335510,tp,tp,tp,tp,fp+fn,fp+fn,fp,fp+fn
80,99980.txt,[309 326 385 367],left/bottom/right/top,[255 206 210 263],top/left/bottom/right,0.7285,3116.0,0.000000,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fn,fp+fn
80,99980.txt,None,left/bottom/right/top,[255 206 210 263],top/left/bottom/right,0.7285,3116.0,0.000000,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp,fp+fn
6271,99979.txt,[146 153 200 193],left/bottom/right/top,[196 143 148 202],top/left/bottom/right,0.8135,2160.0,0.762712,tp,tp,tp,tp,tp,tp,tp,tp
4549,99952.txt,[296 296 321 320],left/bottom/right/top,[388 183 289 221],top/left/bottom/right,0.8441,600.0,0.000000,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fn,fp+fn
4549,99952.txt,None,left/bottom/right/top,[388 183 289 221],top/left/bottom/right,0.8441,600.0,0.000000,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp,fp+fn
2241,99940.txt,[281 299 345 326],left/bottom/right/top,[217 274 193 330],top/left/bottom/right,0.4514,1728.0,0.000000,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fp+fn,fn,fp+fn


In [183]:
def get_recall_score (result_csv, iou_thres, conf_thres):

    # gather y_true and y_scores
    #real_conf = result_df[result_df['confidence'] > conf_thres]
    y_true = pd.Series(result_df['gt_bbox'].notnull() == True).values
    y_true = np.where(y_true == True, 1, y_true)
    y_pred = pd.Series((result_df['result@IoU'+str(iou_thres)+'@conf'+str(conf_thres)] == 'tp') | (result_df['result@IoU'+str(iou_thres)+'@conf'+str(conf_thres)] == 'fp')).values
    ratio = sum(y_pred)/sum(y_true)
    recall_score = metrics.recall_score(y_true, y_pred)
    return ratio, recall_score


In [173]:
ratio, recall_score = get_recall_score(result_df_sparse, 0.05, 0.0)
print(ratio)
print(recall_score)

0.8539345711759505
0.7234305923961096


In [184]:
ratio, recall_score = get_recall_score(result_df_sparse, 0.05, 0.3)
print(ratio)
print(recall_score)

0.6482722143864598
0.6283497884344147


In [186]:
ratio, recall_score = get_recall_score(result_df_sparse, 0.05, 0.5)
print(ratio)
print(recall_score)

0.53737658674189
0.5271509167842031


In [64]:
def get_mAP (result_df, iou_thres, conf_thres):
    
    # gather y_true and y_scores
    real_conf = result_df[result_df['confidence'] > conf_thres]
    y_true = pd.Series(real_conf['gt_bbox'].notnull() == True).values
    #y_true = pd.Series((real_conf['result@IoU'+str(iou_thres)] == 'tp')).values
    print(sum(y_true))
    y_true = np.where(y_true == True, 1, y_true)
    y_scores = real_conf['confidence'].values.astype(float)
    
    # calculate average precision, precision, recall, and thresholds
    ap = metrics.average_precision_score(y_true, y_scores) 
    
    return ap

# args df, iou_thres, conf_thres

In [65]:
mAP = get_mAP(result_df, 0.05, 0.0)
print(mAP)

5655
0.9769034728679117
